# C10でFine-tuningしたViTモデルのフォワードパスを実行し，C10を予測する

## ライブラリのインポート

In [13]:
import os, sys, math
sys.path.append("../src")
import numpy as np
import torch
import pickle
from datasets import load_from_disk
from transformers import DefaultDataCollator, ViTForImageClassification, Trainer
from utils.helper import get_device
from utils.vit_util import processor, transforms, compute_metrics
from utils.constant import ViTExperiment

## 初期設定

In [2]:
# デバイス (cuda, or cpu) の取得
device = get_device()
# datasetをロード (初回の読み込みだけやや時間かかる)
dataset_dir = ViTExperiment.DATASET_DIR
cifar10 = load_from_disk(os.path.join(dataset_dir, "c10"))
# 読み込まれた時にリアルタイムで前処理を適用するようにする
cifar10_preprocessed = cifar10.with_transform(transforms)
# バッチごとの処理のためのdata_collator
data_collator = DefaultDataCollator()
# ラベルを示す文字列のlist
labels = cifar10_preprocessed["train"].features["label"].names
# pretrained modelのロード
pretrained_dir = ViTExperiment.OUTPUT_DIR
model = ViTForImageClassification.from_pretrained(pretrained_dir).to(device)
model.eval()
# 学習時の設定をロード
training_args = torch.load(os.path.join(pretrained_dir, "training_args.bin"))
# Trainerオブジェクトの作成
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    train_dataset=cifar10_preprocessed["train"],
    eval_dataset=cifar10_preprocessed["test"],
    tokenizer=processor,
)

Device: cuda


In [15]:
# 少ないサンプルで実験
sample_size = 33
sample_indices = np.random.choice(len(cifar10_preprocessed["test"]), sample_size)
pred = trainer.predict(cifar10_preprocessed["test"].select(sample_indices))
# predをpickleで保存
with open(os.path.join("./", "pred.pkl"), "wb") as f:
    pickle.dump(pred, f)

## 推論の実行

In [3]:
# データセットのサイズとバッチサイズからイテレーション回数を計算
training_args_dict = training_args.to_dict()
train_batch_size = training_args_dict["per_device_train_batch_size"]
eval_batch_size = training_args_dict["per_device_eval_batch_size"]
train_iter = math.ceil(len(cifar10_preprocessed["train"]) / train_batch_size)
eval_iter = math.ceil(len(cifar10_preprocessed["test"]) / eval_batch_size)

# 訓練・テストデータに対する推論の実行
print(f"predict training data... #iter = {train_iter} ({len(cifar10_preprocessed['train'])} samples / {train_batch_size} batches)")
train_pred = trainer.predict(cifar10_preprocessed["train"])
print(f"predict evaluation data... #iter = {eval_iter} ({len(cifar10_preprocessed['test'])} samples / {eval_batch_size} batches)")
test_pred = trainer.predict(cifar10_preprocessed["test"])

predict training data... #iter = 1563 (50000 samples / 32 batches)


predict evaluation data... #iter = 313 (10000 samples / 32 batches)


## 推論結果をnpyで保存する

In [ ]:
# just for check
np.unique(np.array(cifar10["train"]["label"]), return_counts=True)

In [ ]:
train_labels = np.array(cifar10["train"]["label"])
# train_pred.predictions[1]をsoftmax関数に通して確率に変換
train_pred_proba = torch.nn.functional.softmax(torch.tensor(train_pred.predictions[1]), dim=-1)
# train_pred_probaをnumpy配列に変換
train_pred_proba = train_pred_proba.cpu().numpy()
# ラベルごとに違うファイルとして保存
for c in range(len(labels)):
    tgt_proba = train_pred_proba[train_labels == c]
    # train_pred_probaを保存
    np.save(os.path.join(pretrained_dir, "pred_results", f"train_proba_{c}.npy"), tgt_proba)
    print(f"train_proba_{c}.npy ({tgt_proba.shape}) saved")

In [ ]:
# just for check
np.unique(np.array(cifar10["test"]["label"]), return_counts=True)

In [ ]:
test_labels = np.array(cifar10["test"]["label"])

In [ ]:
test_pred_proba = torch.nn.functional.softmax(torch.tensor(test_pred.predictions[1]), dim=-1)
test_pred_proba = test_pred_proba.cpu().numpy()
# ラベルごとに違うファイルとして保存
for c in range(len(labels)):
    tgt_proba = test_pred_proba[test_labels == c]
    print(tgt_proba.shape)
    # train_pred_probaを保存
    np.save(os.path.join(pretrained_dir, "pred_results", f"test_proba_{c}.npy"), tgt_proba)
    print(f"test_proba_{c}.npy ({tgt_proba.shape}) saved")

# C10でFine-tuningしたViTモデルのフォワードパスを実行し，C10Cを予測する

## 初期設定

In [ ]:
# デバイス (cuda, or cpu) の取得
device = get_device()
# datasetをロード (初回の読み込みだけやや時間かかる)
dataset_dir = ViTExperiment.DATASET_DIR
c10c = load_from_disk(os.path.join(dataset_dir, "c10c"))
tmp_key = "zoom_blur"
c10c[tmp_key]

In [ ]:
# 読み込まれた時にリアルタイムで前処理を適用するようにする
cifar10_preprocessed = c10c[tmp_key].with_transform(transforms)
# バッチごとの処理のためのdata_collator
data_collator = DefaultDataCollator()
# ラベルを示す文字列のlist
labels = cifar10_preprocessed.features["label"].names
# pretrained modelのロード
pretrained_dir = ViTExperiment.OUTPUT_DIR
model = ViTForImageClassification.from_pretrained(pretrained_dir).to(device)
model.eval()
# 学習時の設定をロード
training_args = torch.load(os.path.join(pretrained_dir, "training_args.bin"))
# Trainerオブジェクトの作成
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    eval_dataset=cifar10_preprocessed,
    tokenizer=processor,
)

## 推論の実行

In [ ]:
# データセットのサイズとバッチサイズからイテレーション回数を計算
training_args_dict = training_args.to_dict()
eval_batch_size = training_args_dict["per_device_eval_batch_size"]
eval_iter = math.ceil(len(cifar10_preprocessed) / eval_batch_size)

# 推論の実行
print(f"predict evaluation data... #iter = {eval_iter} ({len(cifar10_preprocessed)} samples / {eval_batch_size} batches)")
test_pred = trainer.predict(cifar10_preprocessed)

In [ ]:
test_pred